# Node Embedding

## Use doc2vec to find the node in the Ontology graph from the protein function text

In [2]:
!pip install nltk

In [12]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hbagheri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/hbagheri/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stopset = set(stopwords.words())

## Preprocessing data

#### One
Put name and synonyms together as a document. Find the most similar one with doc2vec




### PRO dataset download here:
- https://bioportal.bioontology.org/ontologies/PR

In [10]:
with open("../gitdata/pro_reasoned.obo") as fp:
    tags = []
    data = []
    each_tag = ""
    each_term = ""
    id2name = {}
    id2def = {}
    for line in fp.readlines():
        if "[Term]" in line:
            if len(each_tag) > 0:
                tags.append(each_tag.strip(" ").strip("\n"))
                data.append(each_term.strip(" ".strip("\n")))
                id2name[each_tag.strip(" ").strip("\n")] = name
                id2synonym[each_tag.strip(" ").strip("\n")] = defi
            each_tag = ""
            each_term = ""
        if "id:" in line[0:3]:
            each_tag += (" " + line.replace("id: ", ""))
        if "name:" in line[0:5]:
            name = line.replace("name: ", "").strip("\n").strip(" ")
            each_term += (" " + line.replace("name: ", "").strip("\n"))
        if "def:" in line[0:4]:
            defi = line.replace("def: ", "").strip("\n").strip(" ")
            each_term += (" " + line.replace("def: ", "").strip("\n"))
#         if "synonym:" in line[0:10]:
#             synonym = line.replace("synonym: ", "").strip("\n").strip(" ")
#             each_term += (" " + line.replace("synonym: ", "").strip("\n"))
print(len(tags))
print(len(data))
# print(id2name)

321481
321481


In [ ]:
# by Hamid: this is the backup code I modified
with open("../gitdata/pro_reasoned.obo") as fp:
    tags = []
    data = []
    each_tag = ""
    each_term = ""
    id2name = {}
    id2synonym = {}
    for line in fp.readlines():
        if "[Term]" in line:
            if len(each_tag) > 0:
                tags.append(each_tag.strip(" ").strip("\n"))
                data.append(each_term.strip(" ".strip("\n")))
                id2name[each_tag.strip(" ").strip("\n")] = name
                id2synonym[each_tag.strip(" ").strip("\n")] = synonym
            each_tag = ""
            each_term = ""
        if "id:" in line[0:3]:
            each_tag += (" " + line.replace("id: ", ""))
        if "name:" in line[0:5]:
            name = line.replace("name: ", "").strip("\n").strip(" ")
            each_term += (" " + line.replace("name: ", "").strip("\n"))
#         if "def:" in line[0:4]:
#             each_term += (" " + line.replace("def: ", "").strip("\n"))
        if "synonym:" in line[0:10]:
            synonym = line.replace("synonym: ", "").strip("\n").strip(" ")
            each_term += (" " + line.replace("synonym: ", "").strip("\n"))
print(len(tags))
print(len(data))
# print(id2name)

#### Two

Regard name and each synonym as a document separately. 

In [58]:
%%time
with open("../data/pro_reasoned.obo") as fp:
    tags = []
    data = []
    each_tag = ""
    each_term = ""
    id2name = {}
    id2synonym = {}
    for line in fp.readlines():
        if "[Term]" in line:
            if len(each_tag) > 0:
                tags.append(each_tag.strip(" ").strip("\n"))
                data.append(each_term.strip(" ".strip("\n")))
                id2name[each_tag.strip(" ").strip("\n")] = name
#                 id2synonym[each_tag.strip(" ").strip("\n")] = synonym
            each_tag = ""
            each_term = ""
        if "id:" in line[0:3]:
            each_tag += (" " + line.replace("id: ", ""))
        if "name:" in line[0:5]:
            name = line.replace("name: ", "").strip("\n").strip(" ")
            each_term += (" " + line.replace("name: ", "").strip("\n"))
#         if "def:" in line[0:4]:
#             each_term += (" " + line.replace("def: ", "").strip("\n"))
        if "synonym:" in line[0:10]:
            synonym = line.replace("synonym: ", "").strip("\n").strip(" ")
#             each_term += (" " + line.replace("synonym: ", "").strip("\n"))
            data.append(synonym.strip(" ").strip("\n"))
            tags.append(each_tag.strip(" ").strip("\n"))
            each_tag = each_tag.strip(" ").strip("\n")
            if each_tag not in id2synonym:
                id2synonym[each_tag] = []
            id2synonym[each_tag].append(synonym)
print(len(tags))
# print(data)
# print(id2synonym)

910313
CPU times: user 2.65 s, sys: 168 ms, total: 2.82 s
Wall time: 2.82 s


In [13]:
%%time
tag_dict = {}
for i in range(len(tags)):
    tag_dict[i] = tags[i]
tagged_data = [TaggedDocument(words=[token for token in word_tokenize(_d.lower()) if token not in stopset and len(token) > 1], 
                              tags=[tags[i]]) for i, _d in enumerate(data)]

CPU times: user 2min 38s, sys: 1.89 s, total: 2min 40s
Wall time: 2min 43s


In [15]:
# for i in tagged_data:
#     print(i)

In [16]:
%%time
max_epochs = 200
vec_size = 64
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.001,
                min_count=2,
                dm =0)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.00002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("../gitdata/pro_split_name_synonym_d2v.model")
print("Model Saved")

/usr/local/lib/python3.6/site-packages/gensim/models/doc2vec.py:319: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

In [61]:
%%time
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("../gitdata/pro_split_name_synonym_d2v.model")

# #to find the vector of a document which is not in training data
# test_data = "telomere maintenance via recombination"
# test_data = [token for token in word_tokenize(test_data.lower()) if token not in stopset and len(token) > 1]
# print(test_data)

# v1 = model.infer_vector(test_data)
# print("V1_infer", v1)

# model.docvecs.most_similar([v1])

/home/yuepei/anaconda3/envs/labNR@3.6/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CPU times: user 609 ms, sys: 51.3 ms, total: 660 ms
Wall time: 659 ms


## Convert protein name to protein id with name2id dictionary

### Load data

nr_tiny is the top 1000 lines of nr file. We use it as a test of our methods. 

In [18]:
%%time
with open("../gitdata/nr_tiny") as fp:
    protein_dataset = []
    for line in fp.readlines():
        if len(line) == 0 or ">" != line[0]:
            continue
        line_key = line.split(" ")[0]
        aLine = line.strip("").strip("\n").split("\x01")
        name_list = [line_key]
        for each_desc in aLine:
            # e.g. of each_desc: 
            # NP_268346.1 30S ribosomal protein S18 [Lactococcus lactis subsp. lactis Il1403]
            #  ** function name * 
            words = each_desc.split(" ")
            
            # name is protein name
            # find the first '[' as the end of name
            indexs = [words.index(i) for i in words if "[" in i]
            if len(indexs) > 0:
                index = indexs[0]
                name = " ".join(words[1:index])
            else:
                name = " ".join(words[1:])
            
            # pre-processing
            name = name.replace(",", "")
            name = name.replace(".", "")
            name = name.lower()
            if "chain" in name and len(name) > 40:
                continue
#             if "hypothetical protein" in name:
#                 continue
            name_list.append(name)  
        protein_dataset.append(name_list)
    print("{} items loaded. ".format(len(protein_dataset)))

183 items loaded. 
CPU times: user 1.73 s, sys: 397 ms, total: 2.13 s
Wall time: 4.45 s


In [20]:
!pip install goatools

     |████████████████████████████████| 15.1 MB 641 kB/s eta 0:00:011
     |████████████████████████████████| 154 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 10.2 MB 2.2 MB/s eta 0:00:01    |███████████▉                    | 3.8 MB 777 kB/s eta 0:00:09
     |████████████████████████████████| 272 kB 294 kB/s eta 0:00:01
     |████████████████████████████████| 99 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 693 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 17.4 MB 1.1 MB/s eta 0:00:01    |████████▋                       | 4.7 MB 2.5 MB/s eta 0:00:06     |██████████████████▉             | 10.3 MB 1.2 MB/s eta 0:00:06
  Installing build dependencies ... done
  Getting 

In [21]:
%%time
%load_ext autoreload
%autoreload 2

from goatools.obo_parser import GODag
godag = GODag("../gitdata/pro_reasoned.obo")

name2id = {}
for item in godag.items():
    name = item[1].name
    name = name.replace(",", "")
    name = name.replace(".", "")
    name = name.lower()
    name2id[name] = item[1].id
    
print("Name2id dict has {} items.".format(len(name2id)))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
../gitdata/pro_reasoned.obo: fmt(1.2) rel(61.0) 316,370 GO Terms
Name2id dict has 315925 items.
CPU times: user 18.9 s, sys: 11.9 s, total: 30.8 s
Wall time: 1min 3s


### finding word embedding similarity with doc2vec

Use [gensim](https://radimrehurek.com/gensim/models/doc2vec.html)

In [22]:
# with mojarity vote
def find_sim_docvec(test_data):
    test_data = [token for token in word_tokenize(test_data.lower()) if token not in stopset and len(token) > 1]
    item_dict = {}
    for i in range(11):
        most_sim_item = model.docvecs.most_similar([model.infer_vector(test_data, steps=200)])[0]
        if most_sim_item[0] not in item_dict:
            item_dict[most_sim_item[0]] = [0, 0.0]
        item_dict[most_sim_item[0]][0] += 1
        item_dict[most_sim_item[0]][1] += most_sim_item[1]
#     print(item_dict)
    most_sim_item = sorted(item_dict.items(), key=lambda x: x[1], reverse=True)[0]
#     print(most_sim_item)
    sim_doc = most_sim_item[0]
    sim = most_sim_item[1][1] / most_sim_item[1][0]
#     print(sim_doc, sim)
    return sim, sim_doc

In [26]:
!pip install progressbar2


In [29]:
%%time
import time
import progressbar

data_length = 1000
# bar = progressbar.ProgressBar(maxval=data_length)

sim_dict = {}
protein_ids = {}

total_num = 0
cover_num = 0

jac_sim = 0

for index, names in enumerate(protein_dataset[0:data_length]):
    pro_ids = {}
#     bar.update(index)
    for name in names[1:]:
#         print(name)
        name = name.replace("recname: full=", "")
        name = name.replace("MULTISPECIES: ".lower(), "")
        if ";" in name:
            name = name.split(";")[0]
        
        total_num += 1
        if name in name2id:
            pro_id = name2id[name]
            cover_num += 1
            jac_sim = 1
        else:
            if name in sim_dict:
                pro_id, jac_sim = sim_dict[name]
            else:
#                 print(name)
                jac_sim, pro_id = find_sim_docvec(name)
                if pro_id == "":
                    continue
                sim_dict[name] = (pro_id, jac_sim)
        if pro_id not in pro_ids:
            try:
                sim_name = godag[pro_id].name.replace(",", "")
            except:
                sim_name = ""
            try:
                syn_name = " ## ".join(i.replace(",", "") for i in id2synonym[pro_id])
            except:
                syn_name = ""
            pro_ids[pro_id] = [0, name, sim_name, syn_name, jac_sim]
#             pro_ids[pro_id] = [0, name, id2name[pro_id].replace(",", ""), jac_sim]
        pro_ids[pro_id][0] += 1
    protein_ids[names[0]] = pro_ids
fp.close()
print("{} items done. ".format(len(protein_ids)))
print("{0} key_ids, {1} found, {2:0.5f} coverage. ".format(total_num, cover_num, cover_num / float(total_num)))

print("save it in " + "")
# print(protein_ids)

fp = open("../gitdata/pro_doc2vec_sim_4.csv", "w")
for key, items in protein_ids.items():
#     fp.write(key+"\n")
    d = items.items()
    for i in range(len(d)):
        line_string = [key]
        line_string.append(list(d)[i][0])
        line_string.extend(list(d)[i][1])
        fp.write(",".join([str(i) for i in line_string]) + "\n")
fp.close()

183 items done. 
275900 key_ids, 56863 found, 0.20610 coverage. 
save it in 
CPU times: user 2h 21min 15s, sys: 7min 47s, total: 2h 29min 3s
Wall time: 1h 21min 2s


In [30]:
# test_data = "mitochondrion inheritance"
test_data = "uncharacterized protein"
# test_data = [token for token in word_tokenize(test_data.lower()) if token not in stopset and len(token) > 2]
# print(test_data)
# print(model.infer_vector(test_data, steps=200))
# sim_doc = model.docvecs.most_similar([model.infer_vector(test_data)])
sim, sim_doc = find_sim_docvec(test_data)
print(sim_doc)
# find_sim_docvec(test_data)

PR:P64517
